In [16]:
from z3 import *
import numpy as np
from functools import reduce
class f2Poly:
    def __init__(self, terms=None):
        self.terms = set() if terms is None else terms
    def __add__(self, other):
        return f2Poly(self.terms ^ other.terms)
    def __mul__(self, other):
        terms1 = self.terms
        # 情况1：other 是数字（int）
        if isinstance(other, int):
            scalar = other % 2  # 模 2 到 F₂
            if scalar == 0:
                return f2Poly(set())  # 0 * p = 0
            else:
                return self           # 1 * p = p
        # 情况2：other 是 f2Poly
        terms2 = other.terms
        if not terms1 or not terms2:
            return f2Poly(set())  # 0 * anything = 0
        result = set()
        _add = result.add
        _remove = result.remove
        _in = result.__contains__
        for (i1, j1) in terms1:
            for (i2, j2) in terms2:
                term = (i1 + i2, j1 + j2)
                if _in(term):
                    _remove(term)
                else:
                    _add(term)
        return f2Poly(result)
    def __pow__(self, n):
        if n == 0: return f2Poly({(0, 0)})
        if len(self.terms) == 0:  # 零多项式
            if n > 0: return f2Poly(set())
            else: raise ValueError("0 的负幂次无定义")
        if len(self.terms) == 1:  # 单项式
            (i, j) = next(iter(self.terms))
            return f2Poly({(i * n, j * n)})
        # 多项式：快速幂
        if n < 0: raise ValueError("不可逆")
        elif n == 1: return self
        elif n % 2 == 0: return (self * self) ** (n // 2)
        else: return self * (self ** (n - 1))
    def __eq__(self, other):
        if not isinstance(other, f2Poly):
            return False
        return self.terms == other.terms
    def deg(self):
        x_powers = set()
        y_powers = set()
        if self.terms==set():
            x_powers.add(0)
            y_powers.add(0)
        else:
            for i, j in self.terms:
                x_powers.add(i)
                y_powers.add(j)
        return x_powers, y_powers
    def __repr__(self):
        if not self.terms:
            return "0"
        return " + ".join(
            f"x^{i}y^{j}" if i and j else
            f"x^{i}" if i else
            f"y^{j}" if j else
            "1"
            for i, j in self.terms
        )
def mod2(matrix):
    return np.mod(matrix, 2)
def gf2_rref(matrix):
    matrix = mod2(matrix)
    rows, cols = matrix.shape
    row, col = 0, 0
    while row < rows and col < cols:
        if matrix[row, col] == 0:
            for r in range(row + 1, rows):
                if matrix[r, col] == 1:
                    matrix[[row, r]] = matrix[[r, row]]
                    break
        if matrix[row, col] == 1:
            for r in range(rows):
                if r != row and matrix[r, col] == 1:
                    matrix[r] = mod2(matrix[r] + matrix[row])
            row += 1
        col += 1
    return matrix
def antimap(poly):
    inverted_terms = {( -i, -j ) for (i, j) in poly.terms}
    return f2Poly(inverted_terms)
def commute(a, b):
    # 预提取输入
    a0, a1, a2, a3 = a[0][0], a[1][0], a[2][0], a[3][0]
    b0, b1, b2, b3 = b[0][0], b[1][0], b[2][0], b[3][0]
    _antimap = antimap
    _f2Poly = f2Poly
    result = _f2Poly()
    result += _antimap(a0) * b2
    result += _antimap(a1) * b3
    result += _antimap(a2) * b0
    result += _antimap(a3) * b1
    return result
def excimap(*A, oper):
    result = []
    for a in A:
        excitation = commute(a, oper)
        result.append(excitation)
    return result  # 返回 [F2Poly, F2Poly, ...]，就是一个行向量
def tdmap(f, m):
    shifts = [(i, j) for j in range(-m, m + 1) for i in range(-m, m + 1)]
    f_terms_list = [poly.terms for poly in f]
    result = [None] * len(shifts)
    for idx, (i, j) in enumerate(shifts):
        shifted_f = []
        for terms in f_terms_list:
            shifted_terms = {(i + di, j + dj) for (di, dj) in terms}
            shifted_f.append(f2Poly(shifted_terms))
        result[idx] = shifted_f
    return result
def trunmap(f,k):
    ff=np.zeros((2*k+1)**2,dtype=int)
    for (i, j) in f.terms:
        ff[i+k+(j+k)*(2*k+1)]=1
    return ff
def kdeg(f,m):
    a=[]
    b=[]
    for i in range(len(tdmap(f,m)[0])):
        if tdmap(f,m)[0][i]!=f2Poly() and tdmap(f,m)[0][i]!=f2Poly({(0,0)}):
            a.append(max(max(max(tdmap(f,m)[0][i].deg()[0]),-min(tdmap(f,m)[0][i].deg()[0])),
                     max(max(tdmap(f,m)[0][i].deg()[1]),-min(tdmap(f,m)[0][i].deg()[1]))))
    for i in range(len(tdmap(f,m)[-1])):
        if tdmap(f,m)[-1][i]!=f2Poly() and tdmap(f,m)[-1][i]!=f2Poly({(0,0)}):
            b.append(max(max(max(tdmap(f,m)[-1][i].deg()[0]),-min(tdmap(f,m)[-1][i].deg()[0])),
                     max(max(tdmap(f,m)[-1][i].deg()[1]),-min(tdmap(f,m)[-1][i].deg()[1]))))
    return max(max(a),max(b))
def invtrunmap(f,k):
    ff=f2Poly()
    for i in range(len(f)):
        if f[i]!=0:
            a=i%(2*k+1)-k
            b=i//(2*k+1)-k
            ff=ff+f2Poly({(a,b)})
    return ff
def m1(*A,m):
    m1=[]
    m1.extend(tdmap(excimap(*A,oper=x1),m))
    m1.extend(tdmap(excimap(*A,oper=x2),m))
    m1.extend(tdmap(excimap(*A,oper=z1),m))
    m1.extend(tdmap(excimap(*A,oper=z2),m))
    k=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
    k=max(k,m+2)
    a=[]
    for i in range(len(m1)):
        b=[]
        for j in range(len(m1[i])):
            b.extend(trunmap(m1[i][j],k))
        a.append(b)
    return np.array(a)
def m2(*A,m):
    stabdag=[]
    for a in A:
        stabdag.append(np.array(a).T[0])
    for i in range(len(stabdag)):
        for j in range(len(stabdag[i])):
            stabdag[i][j] = antimap(stabdag[i][j])
    M2=[]
    for i in range(len(stabdag)):
        M2.extend(tdmap(stabdag[i],m))
    k=0
    for i in range(len(stabdag)):
       k=max(k,kdeg(stabdag[i],m))
    k=max(k,m+1)
    a=[]
    for i in range(len(M2)):
        b=[]
        for j in range(len(M2[i])):
            b.extend(trunmap(M2[i][j],k))
        a.append(b)
    return np.array(a)
def to(*A,m,mdag):
    m1i=np.hstack((m1(*A,m=m),np.eye(len(m1(*A,m=m)))))
    gem1i=mod2(gf2_rref(m1i))
    gem1,gei=np.hsplit(gem1i,[m1(*A,m=m).shape[1]])
    zeroidx=[]
    for i in range(len(gem1)):
        if all(x==0 for x in gem1[i]):
            zeroidx.append(i)
    nonzerorow,zerorow=np.vsplit(gei,[zeroidx[0]])
    zerorowx,zerorowz=np.hsplit(zerorow,[int(zerorow.shape[1]/2)])
    zerorowx1,zerorowx2=np.hsplit(zerorowx,[int(zerorowx.shape[1]/2)])
    zerorowz1,zerorowz2=np.hsplit(zerorowz,[int(zerorowz.shape[1]/2)])
    o=[]
    for i in range(len(zerorow)):
        oo=[]
        oo.append([invtrunmap(zerorowx1[i],m)])
        oo.append([invtrunmap(zerorowx2[i],m)])
        oo.append([invtrunmap(zerorowz1[i],m)])
        oo.append([invtrunmap(zerorowz2[i],m)])
        o.append(oo)
    o=np.array(o)
    odag=[]
    kk=max(int((np.sqrt(m2(*A,m=mdag).shape[1]/len(s1))-1)/2),mdag+1)
    for i in range(len(o)):
        oodag=[]
        for j in range(len(o[i])):
            oodag.extend(trunmap(antimap(o[i][j][0]),kk))
        odag.append(oodag)
    odag=np.array(odag)
    m2rank=np.sum(np.any(mod2(gf2_rref(m2(*A,m=mdag))), axis=1))
    n=0
    z=0
    for i in range(len(odag)):
        s=np.vstack((m2(*A,m=mdag),odag[i]))
        n=n+1
        if np.sum(np.any(mod2(gf2_rref(s)), axis=1))==m2rank:
            z=z+1
    if n==z:
        return True
    else:
        return False
def m3(*A,m,n,va):
    m1=[]
    m1.extend(tdmap(excimap(*A,oper=x1),m))
    m1.extend(tdmap(excimap(*A,oper=x2),m))
    m1.extend(tdmap(excimap(*A,oper=z1),m))
    m1.extend(tdmap(excimap(*A,oper=z2),m))
    k=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
    if va=='x':
        for i in range(len(A)):
            VV=[]
            for j in range(len(A)):
                if i==j:
                    VV.append(f2Poly({(0,0),(n,0)}))
                else:
                    VV.append(f2Poly())
            m1.extend(tdmap(VV,m))
            k=max(kdeg(VV,m),k)
    elif va=='y':
        for i in range(len(A)):
            VV=[]
            for j in range(len(A)):
                if i==j:
                    VV.append(f2Poly({(0,0),(0,-n)}))
                else:
                    VV.append(f2Poly())
            m1.extend(tdmap(VV,m))
            k=max(kdeg(VV,m),k)
    k=max(k,m+2)
    a=[]
    for i in range(len(m1)):
        b=[]
        for j in range(len(m1[i])):
            b.extend(trunmap(m1[i][j],k))
        a.append(b)
    return np.array(a)
def anyon(*A,m,N,va):
    a=[]
    zero=f2Poly()
    e=[]
    for i in range(len(A)):
        e.append(zero)
    ee=[]
    for i in range(4):
        ee.append(zero)
    for n in range(N):
        m3i=np.hstack((m3(*A,m=m,n=n,va=va),np.eye(len(m3(*A,m=m,n=n,va=va)))))
        gem3i=mod2(gf2_rref(m3i))
        gem3,gei=np.hsplit(gem3i,[m3(*A,m=m,n=n,va=va).shape[1]])
        zeroidx=[]
        for i in range(len(gem3)):
            if all(x==0 for x in gem3[i]):
                zeroidx.append(i)
        nonzerogeii,zerogeii=np.vsplit(gei,[zeroidx[0]])
        geie,geiv=np.hsplit(zerogeii,[(2*m+1)**2*4])
        geivv=np.array(np.hsplit(geiv,len(A)))
        p=[]
        p.append(e)
        for i in range(len(geivv[0])):
            p.append([invtrunmap(geivv[0][i],m),invtrunmap(geivv[1][i],m)])
        u=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
        u=max(u,m+2)
        for i in range(len(p)):
            for j in range(i+1,len(p)):
                s=[]
                for k in range(len(p[i])):
                    s.append(p[i][k]+p[j][k])
                c=[]
                for l in range(len(s)):
                    c.extend(trunmap(s[l],u))
                c=np.array(c)
                m1v=np.vstack((m1(*A,m=m),[c]))
                if np.sum(np.any(mod2(gf2_rref(m1v)), axis=1))==np.sum(np.any(mod2(gf2_rref(m1(*A,m=m))), axis=1)):
                    p[j]=e
        p = [x for x in p if x != e]
        o=[]
        for i in range(len(p)):
            t=[]
            for j in range(len(p[i])):
                t.extend(trunmap(p[i][j],u))
            o.append(t)
        m1vv=m1(*A,m=m)
        for i in range(len(o)):
            rank=np.sum(np.any(mod2(gf2_rref(m1vv))))
            m1vv=np.vstack((m1vv,o[i]))
            if np.sum(np.any(mod2(gf2_rref(m1vv)), axis=1))==rank:
                p[i]=e
        any=np.array([x for x in p if x != e])
        a.append(len(any))
    for i in range(len(a)):
        if a[i]==max(a):
            b=i
            break
    n=b
    m3i=np.hstack((m3(*A,m=m,n=n,va=va),np.eye(len(m3(*A,m=m,n=n,va=va)))))
    gem3i=mod2(gf2_rref(m3i))
    gem3,gei=np.hsplit(gem3i,[m3(*A,m=m,n=n,va=va).shape[1]])
    zeroidx=[]
    for i in range(len(gem3)):
        if all(x==0 for x in gem3[i]):
            zeroidx.append(i)
    nonzerogeii,zerogeii=np.vsplit(gei,[zeroidx[0]])
    geie,geiv=np.hsplit(zerogeii,[(2*m+1)**2*4])
    geivv=np.array(np.hsplit(geiv,len(A)))
    geiee=np.array(np.hsplit(geie,4))
    p=[]
    exci=[]
    exci.append(ee)
    f=len(geivv[0])
    g=len(geivv)
    p.append(e)
    for i in range(f):
        pp=[]
        for j in range(g):
            pp.append(invtrunmap(geivv[j][i],m))
        p.append(pp)
    f=len(geiee[0])
    g=len(geiee)
    for i in range(f):
        excit=[]
        for j in range(g):
            excit.append(invtrunmap(geiee[j][i],m))
        exci.append(excit)
    u=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
    u=max(u,m+2)
    for i in range(len(p)):
        for j in range(i+1,len(p)):
            s=[]
            for k in range(len(p[i])):
                s.append(p[i][k]+p[j][k])
            c=[]
            for l in range(len(s)):
                c.extend(trunmap(s[l],u))
            c=np.array(c)
            m1v=np.vstack((m1(*A,m=m),[c]))
            if np.sum(np.any(mod2(gf2_rref(m1v)), axis=1))==np.sum(np.any(mod2(gf2_rref(m1(*A,m=m))), axis=1)):
                p[j]=e
                exci[j]=ee
    p = [x for x in p if x != e]
    exci= [x for x in exci if x != ee]
    o=[]
    for i in range(len(p)):
        t=[]
        for j in range(len(p[i])):
            t.extend(trunmap(p[i][j],u))
        o.append(t)
    m1vv=m1(*A,m=m)
    for i in range(len(o)):
        rank=np.sum(np.any(mod2(gf2_rref(m1vv)),axis=1))
        m1vv=np.vstack((m1vv,o[i]))
        if np.sum(np.any(mod2(gf2_rref(m1vv)), axis=1))==rank:
            p[i]=e
            exci[i]=ee
    any=[x for x in p if x != e]
    exci=[x for x in exci if x != ee]
    k=np.sum(np.any(mod2(gf2_rref(m1vv)), axis=1))-np.sum(np.any(mod2(gf2_rref(m1(*A,m=m))), axis=1))
    exc=[]
    for i in range(len(exci)):
        excc=[]
        for j in range(len(exci[i])):
            excc.append([exci[i][j]])
        exc.append(excc)
    return any,exc,n,k/2
def stringoper(*A,m,N):
    x=anyon(s1,s2,m=m,N=N,va='x')
    y=anyon(s1,s2,m=m,N=N,va='y')
    m=m+max(x[2],y[2])
    nx=x[2]
    ny=-y[2]
    coff=f2Poly({(nx,0),(0,ny)})
    s=[]
    for i in range(len(x[0])):
        vv=[x*coff for x in x[0][i]]
        u=max(max(max(kdeg(excimap(*A,oper=x1),m),kdeg(excimap(*A,oper=x2),m)),kdeg(excimap(*A,oper=z1),m)),kdeg(excimap(*A,oper=z2),m))
        u=max(u,m+2)
        vvf=[]
        for j in range(len(vv)):
            vvf.extend(trunmap(vv[j],u))
        m1v=np.vstack((m1(*A,m=m),[vvf]))
        m1vi=np.hstack((m1v,np.eye(len(m1v))))
        gem1vi=mod2(gf2_rref(m1vi))
        gem1v,gei=np.hsplit(gem1vi,[len(m1v[0])])
        zeroidx=[]
        for l in range(len(gem1v)):
            if all(x==0 for x in gem1v[l]):
                zeroidx.append(l)
        nonzerogei,zerogei=np.vsplit(gei,[zeroidx[0]])
        zerogeie,zerogeiv=np.hsplit(zerogei,[len(zerogei[0])-1])
        for k in range(len(zerogeiv)):
            if zerogeiv[k][0]==1:
                break
        zerogeiex,zerogeiez=np.hsplit(zerogeie,[int(len(zerogeie[0])/2)])
        zerogeiex1,zerogeiex2=np.hsplit(zerogeiex,[int(len(zerogeiex[0])/2)])
        zerogeiez1,zerogeiez2=np.hsplit(zerogeiez,[int(len(zerogeiez[0])/2)])
        ss=[]
        ss.append(invtrunmap(zerogeiex1[k],m)+x[1][i][0][0])
        ss.append(invtrunmap(zerogeiex2[k],m)+x[1][i][1][0])
        ss.append(invtrunmap(zerogeiez1[k],m)+x[1][i][2][0])
        ss.append(invtrunmap(zerogeiez2[k],m)+x[1][i][3][0])
        s.append(ss)
    exc=[]
    for i in range(len(s)):
        excc=[]
        for j in range(len(s[i])):
            excc.append([s[i][j]])
        exc.append(excc)
    return exc,x,y
def spin(x,y,ystr,q):
    nx=x[2]
    ny=y[2]
    coffu1=f2Poly()
    coffu2=f2Poly()
    coffu3=f2Poly()
    for i in range(q):
        coffu1=coffu1+f2Poly({(-(i+1)*nx,0)})
    for i in range(q+1):
        coffu2=coffu2+f2Poly({(0,-i*ny)})
        coffu3=coffu3+f2Poly({(i*nx,0)})
    u1=[]
    u2=[]
    u3=[]
    for i in range(len(x[1])):
        u11=[]
        u22=[]
        u33=[]
        for j in range(len(x[1][i])):
            u11.append([coffu1*x[1][i][j][0]])
            u22.append([coffu2*ystr[i][j][0]])
            u33.append([coffu3*x[1][i][j][0]])
        u1.append(u11)
        u2.append(u22)
        u3.append(u33)
    the=[]
    for i in range(len(u1)):
        coffe=0
        if (0,0) in commute(u1[i],u2[i]).terms:
            coffe=coffe+1
        if (0,0) in commute(u2[i],u3[i]).terms:
            coffe=coffe+1
        if (0,0) in commute(u3[i],u1[i]).terms:
            coffe=coffe+1
        the.append(np.exp(coffe*1j*np.pi))
    u1=[]
    u2=[]
    u3=[]
    for i in range(len(x[1])-1):
        u10=[]
        u20=[]
        u30=[]
        for j in range(i+1,len(x[1])):
            u11=[]
            u22=[]
            u33=[]
            for k in range(len(x[1][i])):
                u11.append([coffu1*(x[1][i][k][0]+x[1][j][k][0])])
                u22.append([coffu2*(ystr[i][k][0]+ystr[j][k][0])])
                u33.append([coffu3*(x[1][i][k][0]+x[1][j][k][0])])
            u10.append(u11)
            u20.append(u22)
            u30.append(u33)
        u1.append(u10)
        u2.append(u20)
        u3.append(u30)
    theta=[]
    for i in range(len(u1)):
        thetai=[]
        for j in range(len(u1[i])):
            coffe=0
            if (0,0) in commute(u1[i][j],u2[i][j]).terms:
                coffe=coffe+1
            if (0,0) in commute(u2[i][j],u3[i][j]).terms:
                coffe=coffe+1
            if (0,0) in commute(u3[i][j],u1[i][j]).terms:
                coffe=coffe+1
            thetai.append(np.exp(coffe*1j*np.pi))
        theta.append(thetai)
    return the,theta
def rearrangeanyon(x,ystr,sp):
    sesp = [0]
    any=[]
    xst=[]
    yst=[]
    while(len(sp[0])!=0):
        antimu=[]
        if sp[0][sesp[0]].real == -1:
            for i in range(len(sp[1][sesp[0]])):
                if (sp[1][sesp[0]][i].real == -1 and sp[0][i+1]==1)or(sp[1][sesp[0]][i].real == 1 and sp[0][i+1]==-1):
                    t=i
                    break
            x[0][0]=[a+b for a,b in zip(x[0][0],x[0][i+1])]
            sp[0][sesp[0]]=sp[0][sesp[0]].real*sp[0][i+1].real*sp[1][sesp[0]][i].real
            for i in range(len(sp[1][sesp[0]])):
                if i<t:
                    sp[1][sesp[0]][i]=sp[1][sesp[0]][i].real*sp[1][i+1][t-i-1].real
                if i==t:
                    sp[1][sesp[0]][i]=sp[1][sesp[0]][i].real*sp[0][t+1].real
                if i>t:
                    sp[1][sesp[0]][i]=sp[1][sesp[0]][i].real*sp[1][t+1][i-t-1].real
        aany = []
        xsstr=[]
        ysstr=[]
        aany.append(x[0][sesp[0]])
        xsstr.append(x[1][sesp[0]])
        ysstr.append(ystr[sesp[0]])
        for i in range(len(sp[1][sesp[0]])):
            if sp[1][sesp[0]][i].real == -1:
                antimu.append(i)
        t=antimu[0]
        aany.append(x[0][sesp[0] + t + 1])
        xsstr.append(x[1][sesp[0] + t + 1])
        ysstr.append(ystr[sesp[0] + t + 1])
        for i in range(1,len(antimu)):
            x[0][antimu[i]+1] = [a+b for a,b in zip(x[0][sesp[0] + t + 1],x[0][antimu[i]+1])]
            sp[0][antimu[i]+1] = sp[0][antimu[i]+1].real*sp[0][sesp[0] + t + 1].real*sp[1][sesp[0] + t + 1][antimu[i]-sesp[0] - t-1].real
            if antimu[i]+1 < len(sp[0])-1:
                for j in range(antimu[i]+1):
                    if j < (sesp[0] + t + 1):
                        sp[1][j][antimu[i]-j] = sp[1][j][antimu[i]-j].real*sp[1][j][sesp[0] + t-j].real
                    if j ==sesp[0] + t + 1:
                        sp[1][j][antimu[i]-j] = sp[1][j][antimu[i]-j].real*sp[0][sesp[0] + t+1].real
                    if j > (sesp[0] + t + 1):
                        sp[1][j][antimu[i]-j] = sp[1][j][antimu[i]-j].real*sp[1][sesp[0] + t + 1][j-sesp[0] - t - 2].real
            if antimu[i]+1 == len(sp[0])-1:
                for j in range(len(sp[1])):
                    if i < (sesp[0] + t + 1):
                        sp[1][j][-1] = sp[1][j][-1].real*sp[1][j][sesp[0] + t-j].real
                    if i ==sesp[0] + t + 1:
                        sp[1][j][-1] = sp[1][j][-1].real*sp[0][sesp[0] + t+1].real
                    if i > (sesp[0] + i + 1):
                        sp[1][j][-1] = sp[1][j][-1].real*sp[1][sesp[0] + t + 1][j-sesp[0] - t - 2].real
        if sp[0][sesp[0] + t + 1].real == -1:
            x[0][sesp[0] + t + 1]=[a+b for a,b in zip(x[0][sesp[0] + t + 1],x[0][sesp[0]])]
            sp[0][sesp[0] + t + 1]=sp[0][sesp[0] + t + 1].real*sp[0][sesp[0]].real*sp[1][sesp[0]][sesp[0] + t].real
            for i in range(sesp[0] + t + 2):
                if i <sesp[0] + t + 1:
                    if i == sesp[0]:
                        sp[1][i][sesp[0] + t -i]=sp[1][i][sesp[0] + t -i].real*sp[0][sesp[0]].real
                    if i != sesp[0]:
                        sp[1][i][sesp[0] + t -i]=sp[1][i][sesp[0] + t -i].real*sp[1][sesp[0]][i-1].real
                if i == sesp[0] + t + 1:
                    for j in range(sp[1][i]):
                        sp[1][i][j]=sp[1][i][j].real*sp[1][sesp[0]][sesp[0] + t+1+j].real
        for i in range(1,sesp[0] + t + 1):
            if sp[1][i][sesp[0] + t -i].real == -1:
                x[0][i]=[a+b for a,b in zip(x[0][i],x[0][sesp[0]])]
                sp[0][i]=sp[0][i].real*sp[0][sesp[0]]*sp[1][sesp[0]][i-1].real
                for j in range(i):
                    if j ==sesp[0]:
                        sp[1][j][i-j-1]=sp[1][j][i-j-1].real*sp[0][sesp[0]].real
                    else:
                        sp[1][j][i-j-1]=sp[1][j][i-j-1].real*sp[1][sesp[0]][j-1].real
                for j in range(len(sp[1][i])):
                    sp[1][i][j]=sp[1][i][j].real*sp[1][sesp[0]][j+i].real
        if sesp[0] + t + 1 < len(sp[1]):
            for i in range(len(sp[1][sesp[0] + t + 1])):
                if sp[1][sesp[0] + t + 1][i].real == -1:
                    x[0][i+sesp[0] + t + 2]=[a+b for a,b in zip(x[0][i+sesp[0] + t + 2],x[0][sesp[0]])]
                    sp[0][i+sesp[0] + t + 2]=sp[0][i+sesp[0] + t + 2].real*sp[0][sesp[0]]*sp[1][sesp[0]][i+sesp[0] + t + 1].real
                    for j in range(i+sesp[0] + t+2 ):
                        if j ==sesp[0]:
                            sp[1][j][i+sesp[0] + t-j+1]=sp[1][j][i+sesp[0] + t-j+1].real*sp[0][sesp[0]].real
                        else:
                            sp[1][j][i+sesp[0] + t-j+1]=sp[1][j][i+sesp[0] + t-j+1].real*sp[1][sesp[0]][j-1].real
        del x[0][sesp[0] + t + 1]
        del x[0][sesp[0]]
        del x[1][sesp[0] + t + 1]
        del x[1][sesp[0]]
        del ystr[sesp[0] + t + 1]
        del ystr[sesp[0]]
        del sp[0][sesp[0] + t + 1]
        del sp[0][sesp[0]]
        for i in range(len(sp[1])):
            if i < (sesp[0] + t + 1):
                del sp[1][i][t + sesp[0] - i]
        if sesp[0] + t + 1 < len(sp[1]):
            del sp[1][sesp[0] + t + 1]
        del sp[1][sesp[0]]
        any.append(aany)
        xst.append(xsstr)
        yst.append(ysstr)
    return any,xst,yst
def commutequ(a, b, init_val):
    indices = ([(i, (i+8)%16 ) for i in range(16)])
    terms = []
    for i, j in indices:
        if j < len(b):  # 确保不会越界
            terms.append(And(a[i], b[j]))
    return reduce(Xor, terms, init_val)
x1=[[f2Poly({(0,0)})],
    [f2Poly()],
    [f2Poly()],
    [f2Poly()]]
x2=[[f2Poly()],
    [f2Poly({(0,0)})],
    [f2Poly()],
    [f2Poly()]]
z1=[[f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0)})],
    [f2Poly()]]
z2=[[f2Poly()],
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0,0)})]]
s1=[
    [f2Poly({(0, 0),(-1,0),(0,1)})],
    [f2Poly({(0, 0),(0,-1),(1,0)})],
    [f2Poly()],
    [f2Poly()]
]
s2=[
    [f2Poly()],
    [f2Poly()],
    [f2Poly({(0, 0),(-1,0),(0,1)})],
    [f2Poly({(0, 0),(0,-1),(1,0)})],
]
y=stringoper(s1,s2,m=2,N=5)

In [17]:
import copy
yy=copy.deepcopy(y)

In [23]:
y=copy.deepcopy(yy)

In [19]:

sp=spin(y[1],y[2],y[0],q=10)
re=rearrangeanyon(y[1],y[0],sp)

In [24]:
print(y[0])

[[[x^2y^-1 + x^4 + x^-2y^-1 + x^4y^-3 + x^4y^3 + x^4y^-4 + x^4y^2 + x^-3y^-4 + x^4y^-2], [x^3y^-5 + x^4 + x^3y^-2 + x^3y^1 + x^5y^-1 + x^5y^-2 + y^-4 + x^-2y^-5 + x^3y^-3 + x^4y^2 + x^3 + y^-5 + x^2y^-5 + x^2y^-2 + x^1y^-4 + x^-1y^-3 + x^4y^-5 + x^3y^-4 + x^5y^-4 + x^5y^2 + y^-3 + x^2y^-3 + x^1y^-5 + x^2 + x^1y^-2 + x^1y^-1 + x^-2y^-4], [0], [0]], [[x^4y^4 + x^2y^-1 + x^4y^-4 + x^4y^5 + x^-5y^-4 + x^4y^-1 + y^-1], [x^3y^-5 + x^4 + x^5y^-2 + x^5y^4 + x^-3y^-3 + x^1y^-3 + x^2y^-4 + x^2y^2 + x^1 + x^-4y^-5 + x^-1y^-5 + x^-2y^-5 + x^-1y^-1 + x^-1y^-2 + x^-2y^-2 + x^4y^-4 + x^4y^2 + x^3 + x^5y^-3 + x^3y^3 + x^-3y^-4 + y^-5 + y^-2 + x^2y^-5 + x^2y^-1 + x^4y^-5 + x^4y^-1 + x^3y^2 + x^4y^1 + x^5y^-4 + x^-3y^-5 + x^4y^4 + y^-3 + 1 + x^2y^-3 + x^1y^-5 + x^1y^1 + x^1y^-2 + x^1y^-1 + x^-4y^-4 + x^-1y^-4], [0], [0]], [[0], [0], [x^2y^-1 + x^4 + x^-2y^-1 + x^4y^-3 + x^4y^3 + x^4y^-4 + x^4y^2 + x^-3y^-4 + x^4y^-2], [x^3y^-5 + x^4 + x^3y^-2 + x^3y^1 + x^5y^-1 + x^5y^-2 + y^-4 + x^-2y^-5 + x^3y^-3 + x^

In [ ]:
k = 16
n = 16
X = [[Bool(f'x{i}_{j}') for j in range(n)] for i in range(k)]
target_add = []
char = [
    [5],
    [1],
    [6],
    [2],
    [9],
    [13],
    [10],
    [14],
[4],
    [8],
    [11,12],
    [15,16]
]
stab = [[3],
        [12],
        [7],
        [16]]
CCchar = [[1, 3],
          [1, 2],
          [5, 6],
          [5, 7],
          [9, 11],
          [9, 10],
          [13, 14],
          [13, 15],
          [4],
          [8],
          [12],
          [16]]
CCstab = [[1, 2, 3, 4],
          [9, 10, 11, 12],
          [5, 6, 7, 8],
          [13, 14, 15, 16]]
charstab = char + stab
s = Solver()
for i in range(len(charstab)):
    q = np.zeros((1, n), dtype=int)
    for j in range(len(charstab[i])):
        q[0][charstab[i][j] - 1] = 1
    target_add.append(q[0])
for i in range(len(char)):
    for j in range(n):
        if len(CCchar[i]) != 1:
            constraint = (Xor(X[CCchar[i][0] - 1][j], X[CCchar[i][1] - 1][j]) == BoolVal(target_add[i][j]))
            s.assert_and_track(constraint, f'c{i}_{j}')
        if len(CCchar[i]) == 1:
            constraint = (X[CCchar[i][0] - 1][j] == BoolVal(target_add[i][j]))
            s.assert_and_track(constraint, f'c{i}_{j}')
for i in range(len(stab)):
    for j in range(n):
        variables = [X[CCstab[i][0] - 1][j], X[CCstab[i][1] - 1][j], X[CCstab[i][2] - 1][j], X[CCstab[i][3] - 1][j]]
        xor_expr = reduce(Xor, variables)
        constraint = (xor_expr == BoolVal(target_add[len(char) + i][j]))
        s.assert_and_track(constraint, f'c{len(char) + i}_{j}')

for i in range(k):
    for j in range(k):
        if j == (i + 8) % 16:
            s.assert_and_track(commutequ(X[i], X[j], init_val=False), f'c{len(charstab) + i}_{j}')
        else:
            s.assert_and_track(commutequ(X[i], X[j], init_val=True), f'c{len(charstab) + i}_{j}')

for i in range(k):
    s.assert_and_track(Not(And([Not(X[i][j]) for j in range(n)])), f'c{len(charstab) + k + i}_{j}')
for i in range(k):
    for j in range(i + 1, k):
        # 向量 X[i] 和 X[j] 不相等
        s.assert_and_track(Not(And([X[i][p] == X[j][p] for p in range(n)])), f'c{len(charstab) + 2 * k + i}_{j}')

# 求解
if s.check() == sat:
    m = s.model()

    def get_vector(model, vec):
        return [1 if is_true(model.eval(v)) else 0 for v in vec]

    sol = []
    for i in range(k):
        sol.append(get_vector(m, X[i]))
    for i in range(k):
        ind = []
        for j in range(len(sol[i])):
            if sol[i][j] != 0:
                ind.append(j + 1)
        print(f'x{i + 1}=', sol[i], ind)
else:
    print("Solver is unsatisfiable. Analyzing which commute constraints are in conflict...")
    s.set(unsat_core=True)
    core = s.unsat_core()
    num_charstab = len(charstab)
    conflict_commute = []
    for c in core:
        name = str(c)
        if name.startswith('c') and '_' in name:
            try:
                parts = name[1:].split('_')
                idx1 = int(parts[0])
                if idx1 >= num_charstab:
                    i = idx1 - num_charstab
                    if i < k:
                        j = int(parts[1])
                        if j < k:
                            conflict_commute.append((i, j))
            except:
                continue
    if conflict_commute:
        print("The following commute constraints are in conflict with other constraints:")
        for i, j in conflict_commute:
            print(f"  commute(X[{i}], X[{j}])")
    else:
        print("No commute constraint is in the unsat core. Conflict comes from non-commute constraints.")


In [16]:
sol = [[1, 3, 4, 5],
       [3, 4, 5],
       [1, 3, 4],
       [4],
       [2, 6, 7, 8],
       [2, 7, 8],
       [6, 7, 8],
       [8],
       [9, 11, 13],
       [9, 11],
       [11, 13],
       [11, 12],
       [10, 14, 15],
       [14, 15],
       [10, 15],
       [15, 16]
       ]
soltion = []
for i in range(len(sol)):
    a = np.zeros((1, 16))[0]
    for j in range(len(sol[i])):
        a[sol[i][j] - 1] = 1
    soltion.append(a.tolist())
print(soltion[12])
for i in range(len(soltion)):
    for j in range(len(soltion)):
        if j == (i + 8) % 16:
            a = [BoolVal(bool(x)) for x in soltion[i]]
            b = [BoolVal(bool(x)) for x in soltion[j]]
            result = commute(a, b, BoolVal(False))
            if simplify(result) == False:
                print(simplify(result), i, j)  # True
        else:
            a = [BoolVal(bool(x)) for x in soltion[i]]
            b = [BoolVal(bool(x)) for x in soltion[j]]
            result = commute(a, b, BoolVal(True))
            if simplify(result) == False:
                print(simplify(result), i, j)  # True

for i in range(len(stab)):
    for j in range(n):
        a = [BoolVal(bool(x)) for x in soltion[CCstab[i][0] - 1]]
        b = [BoolVal(bool(x)) for x in soltion[CCstab[i][1] - 1]]
        c = [BoolVal(bool(x)) for x in soltion[CCstab[i][2] - 1]]
        d = [BoolVal(bool(x)) for x in soltion[CCstab[i][3] - 1]]
        variables = [a[j], b[j], c[j], d[j]]
        xor_expr = reduce(Xor, variables)
        constraint = (xor_expr == BoolVal(target_add[len(char) + i][j]))
        if simplify(constraint) == False:
            print(simplify(constraint))
for i in range(len(char)):
    for j in range(n):
        a = [BoolVal(bool(x)) for x in soltion[CCchar[i][0] - 1]]
        if len(CCchar[i]) != 1:
            b = [BoolVal(bool(x)) for x in soltion[CCchar[i][1] - 1]]
            constraint = (Xor(a[j], b[j]) == BoolVal(target_add[i][j]))
            if simplify(constraint) == False:
                print(simplify(constraint), i, j)
        if len(CCchar[i]) == 1:
            constraint = (a[j] == BoolVal(target_add[i][j]))
            if simplify(constraint) == False:
                print(simplify(constraint), i, j)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
